<a href="https://colab.research.google.com/github/jmarrietar/ocular/blob/master/notebooks/PAWS_DR_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TO DO: 
Update snippets of this code in order to make it accordance to branch feature/DR-images-v2

In [ ]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir ./

Cloning into 'apex'...
remote: Enumerating objects: 8054, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 8054 (delta 68), reused 101 (delta 47), pack-reused 7913
Receiving objects: 100% (8054/8054), 14.11 MiB | 13.92 MiB/s, done.
Resolving deltas: 100% (5467/5467), done.
/content/apex
Created temporary directory: /tmp/pip-ephem-wheel-cache-uw47qh37
Created temporary directory: /tmp/pip-req-tracker-zpz831xk
Created requirements tracker '/tmp/pip-req-tracker-zpz831xk'
Created temporary directory: /tmp/pip-install-afwoo_0b
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-ojkwxo_q
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-zpz831xk'
    Running setup.py (path:/tmp/pip-req-build-ojkwxo_q/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info


    torch.__version__  = 1.9.0+cu102


    running egg_info
    creating /tmp/pip-r

In [ ]:
cd ..

/content


In [ ]:
!pip install -U PyYAML

     |████████████████████████████████| 645kB 4.3MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!git clone -b feature/DR-images https://github.com/jmarrietar/suncet.git

Cloning into 'suncet'...
remote: Enumerating objects: 207, done.
remote: Counting objects: 100% (207/207), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 207 (delta 111), reused 162 (delta 72), pack-reused 0
Receiving objects: 100% (207/207), 1.09 MiB | 5.78 MiB/s, done.
Resolving deltas: 100% (111/111), done.


In [ ]:
cd suncet

/content/suncet


In [ ]:
!mkdir datasets

In [ ]:
!mkdir datasets/dr

In [ ]:
!mkdir logs

In [ ]:
!python download.py -d sample@1000

Downloading...
From: https://drive.google.com/uc?id=1DPZrHrj3Bdte5Dc6NCZ33CAqMG-Oipa2
To: /content/suncet/datasets/dr/sample@1000.zip
108MB [00:01, 77.1MB/s]


In [ ]:
!python3 main.py --sel paws_train --fname configs/paws/dr_train.yaml

INFO:root:called-params paws_train configs/paws/dr_train.yaml
INFO:root:loaded params...
{   'criterion': {   'classes_per_batch': 2,
                     'me_max': True,
                     'sharpen': 0.25,
                     'supervised_epochs': 10,
                     'supervised_imgs_per_class': 32,
                     'supervised_views': 1,
                     'temperature': 0.1,
                     'unsupervised_batch_size': 32},
    'data': {   'color_jitter_strength': 1.0,
                'data_seed': None,
                'dataset': 'dr',
                'image_folder': 'dr/sample@1000/',
                'label_smoothing': 0.1,
                'multicrop': 0,
                'normalize': True,
                'root_path': 'datasets/',
                'subset_path': 'dr_subsets',
                'unique_classes_per_rank': False,
                'unlabeled_frac': 0.9},
    'logging': {'folder': 'logs/', 'write_tag': 'paws'},
    'meta': {   'copy_data': True,
            

TO DO: Averiguar esto del Multicrop y por que PAWS funciono funciono luego de que lo cambiara a zero?. 

In [ ]:
###################################
########### DEBUG #################
###################################

In [ ]:
# Copyright (c) Facebook, Inc. and its affiliates.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#

import os

# -- FOR DISTRIBUTED TRAINING ENSURE ONLY 1 DEVICE VISIBLE PER PROCESS
try:
    # -- WARNING: IF DOING DISTRIBUTED TRAINING ON A NON-SLURM CLUSTER, MAKE
    # --          SURE TO UPDATE THIS TO GET LOCAL-RANK ON NODE, OR ENSURE
    # --          THAT YOUR JOBS ARE LAUNCHED WITH ONLY 1 DEVICE VISIBLE
    # --          TO EACH PROCESS
    os.environ['CUDA_VISIBLE_DEVICES'] = os.environ['SLURM_LOCALID']
except Exception:
    pass

import logging
import sys
from collections import OrderedDict

import numpy as np

import torch

import src.resnet as resnet
import src.wide_resnet as wide_resnet
from src.utils import (
    gpu_timer,
    init_distributed,
    WarmupCosineSchedule,
    CSVLogger,
    AverageMeter
)
from src.losses import (
    init_simclr_loss,
    init_suncet_loss,
    make_labels_matrix
)
from src.data_manager import (
    init_data,
    make_transforms
)
from src.sgd import SGD
from src.lars import LARS

import apex
from torch.nn.parallel import DistributedDataParallel

In [ ]:
import argparse

import torch.multiprocessing as mp

import pprint
import yaml

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument(
    '--fname', type=str,
    help='name of config file to load',
    default='configs.yaml')
parser.add_argument(
    '--devices', type=str, nargs='+', default=['cuda:0'],
    help='which devices to use on local machine')
parser.add_argument(
    '--sel', type=str,
    help='which script to run',
    choices=[
        'paws_train',
        'suncet_train',
        'fine_tune',
        'snn_fine_tune'
    ])

_StoreAction(option_strings=['--sel'], dest='sel', nargs=None, const=None, default=None, type=<class 'str'>, choices=['paws_train', 'suncet_train', 'fine_tune', 'snn_fine_tune'], help='which script to run', metavar=None)

In [ ]:
args = parser.parse_args(['--sel', 'paws_train',
                            '--fname', 'configs/paws/dr_train.yaml'
])

In [ ]:
fname = args.fname
sel = args.sel

In [ ]:
import logging
logging.basicConfig()
logger = logging.getLogger()

logger.info(f'called-params {sel} {fname}')

In [ ]:
rank = 0

In [ ]:
# -- load script params
params = None
with open(fname, 'r') as y_file:
    params = yaml.load(y_file, Loader=yaml.FullLoader)
    logger.info('loaded params...')
    if rank == 0:
        pp = pprint.PrettyPrinter(indent=4)
        pp.pprint(params)

if rank == 0:
    dump = os.path.join(params['logging']['folder'], f'params-{sel}.yaml')
    with open(dump, 'w') as f:
        yaml.dump(params, f)

{   'criterion': {   'classes_per_batch': 2,
                     'me_max': True,
                     'sharpen': 0.25,
                     'supervised_epochs': 10,
                     'supervised_imgs_per_class': 32,
                     'supervised_views': 2,
                     'temperature': 0.1,
                     'unsupervised_batch_size': 64},
    'data': {   'color_jitter_strength': 1.0,
                'data_seed': None,
                'dataset': 'dr',
                'image_folder': 'dr/sample@1000/',
                'label_smoothing': 0.1,
                'multicrop': 6,
                'normalize': True,
                'root_path': 'datasets/',
                'subset_path': 'dr_subsets',
                'unique_classes_per_rank': False,
                'unlabeled_frac': 0.9},
    'logging': {'folder': 'logs/', 'write_tag': 'paws'},
    'meta': {   'copy_data': True,
                'device': 'cuda:0',
                'load_checkpoint': False,
                'model_

In [ ]:
args = params

# PAWS TRAIN

In [ ]:
def init_model(
    device,
    model_name='resnet50',
    use_pred=False,
    output_dim=128
):
    if 'wide_resnet' in model_name:
        encoder = wide_resnet.__dict__[model_name](dropout_rate=0.0)
        hidden_dim = 128
    else:
        encoder = resnet.__dict__[model_name]()
        hidden_dim = 2048
        if 'w2' in model_name:
            hidden_dim *= 2
        elif 'w4' in model_name:
            hidden_dim *= 4

    # -- projection head
    encoder.fc = torch.nn.Sequential(OrderedDict([
        ('fc1', torch.nn.Linear(hidden_dim, hidden_dim)),
        ('bn1', torch.nn.BatchNorm1d(hidden_dim)),
        ('relu1', torch.nn.ReLU(inplace=True)),
        ('fc2', torch.nn.Linear(hidden_dim, hidden_dim)),
        ('bn2', torch.nn.BatchNorm1d(hidden_dim)),
        ('relu2', torch.nn.ReLU(inplace=True)),
        ('fc3', torch.nn.Linear(hidden_dim, output_dim))
    ]))

    # -- prediction head
    encoder.pred = None
    if use_pred:
        mx = 4  # 4x bottleneck prediction head
        pred_head = OrderedDict([])
        pred_head['bn1'] = torch.nn.BatchNorm1d(output_dim)
        pred_head['fc1'] = torch.nn.Linear(output_dim, output_dim//mx)
        pred_head['bn2'] = torch.nn.BatchNorm1d(output_dim//mx)
        pred_head['relu'] = torch.nn.ReLU(inplace=True)
        pred_head['fc2'] = torch.nn.Linear(output_dim//mx, output_dim)
        encoder.pred = torch.nn.Sequential(pred_head)

    encoder.to(device)
    logger.info(encoder)
    return encoder

In [ ]:
def init_opt(
    encoder,
    iterations_per_epoch,
    start_lr,
    ref_lr,
    ref_mom,
    nesterov,
    warmup,
    num_epochs,
    weight_decay=1e-6,
    final_lr=0.0
):
    param_groups = [
        {'params': (p for n, p in encoder.named_parameters()
                    if ('bias' not in n) and ('bn' not in n))},
        {'params': (p for n, p in encoder.named_parameters()
                    if ('bias' in n) or ('bn' in n)),
         'LARS_exclude': True,
         'weight_decay': 0}
    ]
    optimizer = SGD(
        param_groups,
        weight_decay=weight_decay,
        momentum=0.9,
        nesterov=nesterov,
        lr=ref_lr)
    scheduler = WarmupCosineSchedule(
        optimizer,
        warmup_steps=warmup*iterations_per_epoch,
        start_lr=start_lr,
        ref_lr=ref_lr,
        final_lr=final_lr,
        T_max=num_epochs*iterations_per_epoch)
    optimizer = LARS(optimizer, trust_coefficient=0.001)
    return encoder, optimizer, scheduler

In [ ]:
import os

# -- FOR DISTRIBUTED TRAINING ENSURE ONLY 1 DEVICE VISIBLE PER PROCESS
try:
    # -- WARNING: IF DOING DISTRIBUTED TRAINING ON A NON-SLURM CLUSTER, MAKE
    # --          SURE TO UPDATE THIS TO GET LOCAL-RANK ON NODE, OR ENSURE
    # --          THAT YOUR JOBS ARE LAUNCHED WITH ONLY 1 DEVICE VISIBLE
    # --          TO EACH PROCESS
    os.environ['CUDA_VISIBLE_DEVICES'] = os.environ['SLURM_LOCALID']
except Exception:
    pass

import logging
import sys
from collections import OrderedDict

import numpy as np

import torch

import src.resnet as resnet
import src.wide_resnet as wide_resnet
from src.utils import (
    gpu_timer,
    init_distributed,
    WarmupCosineSchedule,
    CSVLogger,
    AverageMeter
)
from src.losses import (
    init_paws_loss,
    make_labels_matrix
)
from src.data_manager import (
    init_data,
    make_transforms,
    make_multicrop_transform
)
from src.sgd import SGD
from src.lars import LARS

import apex
from torch.nn.parallel import DistributedDataParallel

# --
log_timings = True
log_freq = 10
checkpoint_freq = 50
# --

_GLOBAL_SEED = 0
np.random.seed(_GLOBAL_SEED)
torch.manual_seed(_GLOBAL_SEED)
torch.backends.cudnn.benchmark = True

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger()

In [ ]:
# ----------------------------------------------------------------------- #
#  PASSED IN PARAMS FROM CONFIG FILE
# ----------------------------------------------------------------------- #
# -- META
model_name = args['meta']['model_name']
output_dim = args['meta']['output_dim']
load_model = args['meta']['load_checkpoint']
r_file = args['meta']['read_checkpoint']
copy_data = args['meta']['copy_data']
use_fp16 = args['meta']['use_fp16']
use_pred_head = args['meta']['use_pred_head']
device = torch.device(args['meta']['device'])
torch.cuda.set_device(device)

# -- CRITERTION
reg = args['criterion']['me_max']
supervised_views = args['criterion']['supervised_views']
classes_per_batch = args['criterion']['classes_per_batch']
s_batch_size = args['criterion']['supervised_imgs_per_class']
u_batch_size = args['criterion']['unsupervised_batch_size']
temperature = args['criterion']['temperature']
sharpen = args['criterion']['sharpen']

# -- DATA
unlabeled_frac = args['data']['unlabeled_frac']
color_jitter = args['data']['color_jitter_strength']
normalize = args['data']['normalize']
root_path = args['data']['root_path']
image_folder = args['data']['image_folder']
dataset_name = args['data']['dataset']
subset_path = args['data']['subset_path']
unique_classes = args['data']['unique_classes_per_rank']
multicrop = args['data']['multicrop']
label_smoothing = args['data']['label_smoothing']
data_seed = None
if 'cifar10' in dataset_name:
    data_seed = args['data']['data_seed']
    crop_scale = (0.75, 1.0) if multicrop > 0 else (0.5, 1.0)
    mc_scale = (0.3, 0.75)
    mc_size = 18
else:
    crop_scale = (0.14, 1.0) if multicrop > 0 else (0.08, 1.0)
    mc_scale = (0.05, 0.14)
    mc_size = 96

In [ ]:
# -- OPTIMIZATION
wd = float(args['optimization']['weight_decay'])
num_epochs = args['optimization']['epochs']
warmup = args['optimization']['warmup']
start_lr = args['optimization']['start_lr']
lr = args['optimization']['lr']
final_lr = args['optimization']['final_lr']
mom = args['optimization']['momentum']
nesterov = args['optimization']['nesterov']

# -- LOGGING
folder = args['logging']['folder']
tag = args['logging']['write_tag']

In [ ]:
label_smoothing

0.1

In [ ]:
world_size = 1
rank = 0 

In [ ]:
# -- log/checkpointing paths
log_file = os.path.join(folder, f'{tag}_r{rank}.csv')
save_path = os.path.join(folder, f'{tag}' + '-ep{epoch}.pth.tar')
latest_path = os.path.join(folder, f'{tag}-latest.pth.tar')
best_path = os.path.join(folder, f'{tag}' + '-best.pth.tar')
load_path = None
if load_model:
    load_path = os.path.join(folder, r_file) if r_file is not None else latest_path

In [ ]:
# -- make csv_logger
csv_logger = CSVLogger(log_file,
                        ('%d', 'epoch'),
                        ('%d', 'itr'),
                        ('%.5f', 'paws-xent-loss'),
                        ('%.5f', 'paws-me_max-reg'),
                        ('%d', 'time (ms)'))

In [ ]:
# -- init model
encoder = init_model(
    device=device,
    model_name=model_name,
    use_pred=use_pred_head,
    output_dim=output_dim)
if world_size > 1:
    process_group = apex.parallel.create_syncbn_process_group(0)
    encoder = apex.parallel.convert_syncbn_model(encoder, process_group=process_group)

In [ ]:
# -- init losses
paws = init_paws_loss(
    multicrop=multicrop,
    tau=temperature,
    T=sharpen,
    me_max=reg)

In [ ]:
# -- assume support images are sampled with ClassStratifiedSampler
labels_matrix = make_labels_matrix(
    num_classes=classes_per_batch,
    s_batch_size=s_batch_size,
    world_size=world_size,
    device=device,
    unique_classes=unique_classes,
    smoothing=label_smoothing)

In [ ]:
print("normalize {}".format(normalize))

normalize True


In [ ]:
# -- make data transforms
transform, init_transform = make_transforms(
    dataset_name=dataset_name,
    subset_path=subset_path,
    unlabeled_frac=unlabeled_frac,
    training=True,
    split_seed=data_seed,
    crop_scale=crop_scale,
    basic_augmentations=False,
    color_jitter=color_jitter,
    normalize=normalize)
multicrop_transform = (multicrop, None)
if multicrop > 0:
    multicrop_transform = make_multicrop_transform(
            dataset_name=dataset_name,
            num_crops=multicrop,
            size=mc_size,
            crop_scale=mc_scale,
            normalize=normalize,
            color_distortion=color_jitter)

In [ ]:
# -- init data-loaders/samplers
(unsupervised_loader,
    unsupervised_sampler,
    supervised_loader,
    supervised_sampler) = init_data(
        dataset_name=dataset_name,
        transform=transform,
        init_transform=init_transform,
        supervised_views=supervised_views,
        u_batch_size=u_batch_size,
        s_batch_size=s_batch_size,
        unique_classes=unique_classes,
        classes_per_batch=classes_per_batch,
        multicrop_transform=multicrop_transform,
        world_size=world_size,
        rank=rank,
        root_path=root_path,
        image_folder=image_folder,
        training=True,
        copy_data=copy_data)
iter_supervised = None
ipe = len(unsupervised_loader)
logger.info(f'iterations per epoch: {ipe}')

self.multicrop_transform (0, None)
self.multicrop_transform (0, None)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# -- init optimizer and scheduler
scaler = torch.cuda.amp.GradScaler(enabled=use_fp16)
encoder, optimizer, scheduler = init_opt(
    encoder=encoder,
    weight_decay=wd,
    start_lr=start_lr,
    ref_lr=lr,
    final_lr=final_lr,
    ref_mom=mom,
    nesterov=nesterov,
    iterations_per_epoch=ipe,
    warmup=warmup,
    num_epochs=num_epochs)
if world_size > 1:
    encoder = DistributedDataParallel(encoder, broadcast_buffers=False)

In [ ]:
start_epoch = 0
# -- load training checkpoint
if load_model:
    encoder, optimizer, start_epoch = load_checkpoint(
        r_path=load_path,
        encoder=encoder,
        opt=optimizer,
        scaler=scaler,
        use_fp16=use_fp16)
    for _ in range(start_epoch):
        for _ in range(ipe):
            scheduler.step()

In [ ]:
################## DEBUG #################

In [ ]:
epoch = 0 

In [ ]:
# -- update distributed-data-loader epoch
unsupervised_sampler.set_epoch(epoch)
if supervised_sampler is not None:
    supervised_sampler.set_epoch(epoch)

loss_meter = AverageMeter()
ploss_meter = AverageMeter()
rloss_meter = AverageMeter()
time_meter = AverageMeter()
data_meter = AverageMeter()

In [ ]:
best_loss = None

In [ ]:
for itr, udata in enumerate(unsupervised_loader):

    def load_imgs():
        # -- unsupervised imgs
        uimgs = [u.to(device, non_blocking=True) for u in udata[:-1]]
        # -- supervised imgs
        global iter_supervised
        try:
            sdata = next(iter_supervised)
        except Exception:
            iter_supervised = iter(supervised_loader)
            logger.info(f'len.supervised_loader: {len(iter_supervised)}')
            sdata = next(iter_supervised)
        finally:
            labels = torch.cat([labels_matrix for _ in range(supervised_views)])
            simgs = [s.to(device, non_blocking=True) for s in sdata[:-1]]
        # -- concatenate supervised imgs and unsupervised imgs
        imgs = simgs + uimgs
        return imgs, labels
    (imgs, labels), dtime = gpu_timer(load_imgs)
    data_meter.update(dtime)
    break

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
from logging import getLogger
import torch
from src.utils import (
    AllGather,
    AllReduce
)

In [ ]:
tau=0.1
T=0.25
me_max=True

In [ ]:
multicrop

6

In [ ]:
len(imgs)

4

In [ ]:
with torch.cuda.amp.autocast(enabled=use_fp16):
    optimizer.zero_grad()

    # --
    # h: representations of 'imgs' before head
    # z: representations of 'imgs' after head
    # -- If use_pred_head=False, then encoder.pred (prediction
    #    head) is None, and _forward_head just returns the
    #    identity, z=h
    h, z = encoder(imgs, return_before_head=True)

    # Compute paws loss in full precision
    with torch.cuda.amp.autocast(enabled=False):

        # Step 1. convert representations to fp32
        h, z = h.float(), z.float()

        # Step 2. determine anchor views/supports and their
        #         corresponding target views/supports
        # --
        num_support = supervised_views * s_batch_size * classes_per_batch
        # --
        anchor_supports = z[:num_support]
        anchor_views = z[num_support:]
        # --
        target_supports = h[:num_support].detach()
        target_views = h[num_support:].detach()
        target_views = torch.cat([
            target_views[u_batch_size:2*u_batch_size],
            target_views[:u_batch_size]], dim=0)

        softmax = torch.nn.Softmax(dim=1)

        def sharpen(p):
            sharp_p = p**(1./T)
            sharp_p /= torch.sum(sharp_p, dim=1, keepdim=True)
            return sharp_p

        def snn(query, supports, labels):
            """ Soft Nearest Neighbours similarity classifier """
            # Step 1: normalize embeddings
            query = torch.nn.functional.normalize(query)
            supports = torch.nn.functional.normalize(supports)

            # Step 2: gather embeddings from all workers
            supports = AllGather.apply(supports)

            # Step 3: compute similarlity between local embeddings
            return softmax(query @ supports.T / tau) @ labels


        #anchor_views=anchor_views
        #anchor_supports=anchor_supports
        anchor_support_labels=labels
        #target_views=target_views
        #target_supports=target_supports
        target_support_labels=labels
        # -- NOTE: num views of each unlabeled instance = 2+multicrop
        batch_size = len(anchor_views) // (2+multicrop)

        # Step 1: compute anchor predictions
        probs = snn(anchor_views, anchor_supports, anchor_support_labels)


        # Step 2: compute targets for anchor predictions
        with torch.no_grad():
            targets = snn(target_views, target_supports, target_support_labels)
            targets = sharpen(targets)
            if multicrop > 0:
                mc_target = 0.5*(targets[:batch_size]+targets[batch_size:])
                targets = torch.cat([targets, *[mc_target for _ in range(multicrop)]], dim=0)
            targets[targets < 1e-4] *= 0  # numerical stability

        # Step 3: compute cross-entropy loss H(targets, queries)
        loss = torch.mean(torch.sum(torch.log(probs**(-targets)), dim=1))

        # Step 4: compute me-max regularizer
        rloss = 0.
        if me_max:
            avg_probs = AllReduce.apply(torch.mean(sharpen(probs), dim=0))
            rloss -= torch.sum(torch.log(avg_probs**(-avg_probs)))

RuntimeError: ignored

In [ ]:
multicrop

6

In [ ]:
batch_size

32

In [ ]:
num_support

64

In [ ]:
loss

tensor(0.6944, device='cuda:0', grad_fn=<MeanBackward0>)

In [ ]:
batch_size

8